# Generalizations of Max-Flow

The purpose of this segment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize ourselves with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 2

To implement reduction we will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. we are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [3]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    g = nx.DiGraph()
    f = open(infile)
    # For temp node name generation:
    temp = 0
    for line in f:
        data = line.strip().split(' ')
        if data[0] == 'n':
            g.add_node(data[1], demand=int(data[2]))
            
        elif data[0] == 'a':
            # Set node demands to 0:
            if not data[1] in g:
                g.add_node(data[1], demand=0)
            if not data[2] in g:
                g.add_node(data[2], demand=0)
            
            # Add edge:
            if not g.has_edge(data[1], data[2]):
                g.add_edge(data[1], data[2], capacity=int(data[4]), weight=int(data[5]))
               
            # Reduction step - add an extra node if edge already exists:
            else:
                temp_name = 'temp_' + str(temp)
                g.add_node(temp_name, demand=0)
                g.add_edge(data[1], temp_name, capacity=int(data[4]), weight=0)
                g.add_edge(temp_name, data[2], capacity=int(data[4]), weight=int(data[5]))
                temp = temp + 1
    return(g)

The following will check that your code outputs the expected min cost flow values on several test instances.

In [4]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [5]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    prob = pulp.LpProblem("Min Cost Flow", pulp.LpMinimize)

    # Create list of edges to feed to LpVariable.dicts
    # Var names are based on names of connected nodes
    edges = [str(e[0]) + ',' + str(e[1]) for e in G.edges()]
    
    # Variables:
    flow_vars = pulp.LpVariable.dicts("flow", edges, 0)
    
    # Objective function:
    prob += pulp.lpSum([ e[2]['weight'] * flow_vars[str(e[0]) + ',' + str(e[1])] for e in G.edges(data=True) ])
    
    # Flows must be less than capacities:
    for e in G.edges(data=True):
        prob += flow_vars[str(e[0]) + ',' + str(e[1])] <= e[2]['capacity']
    
    # (Flow out - flow in) must equal demand of node:
    for n in G.nodes(data=True):
        prob += (
            pulp.lpSum(     [flow_vars[ str(e[0]) + ',' + str(e[1]) ] for e in G.out_edges(n[0])] )     
            - pulp.lpSum(   [flow_vars[ str(e[0]) + ',' + str(e[1]) ] for e in G.in_edges(n[0])]  )    
            == n[1]['demand']
        )
    
    
    # Any more requiremens...?
    
    prob.solve()
    return int(pulp.value(prob.objective))

The following will check that the LP finds the same optimal values as previously.

In [6]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
